# Introduction to AI Agents using OpenAI Agents SDK
## Complete Guide to GPT-5.2 & GPT-4o Agents (2026 Edition)

> ⚠️ **COST WARNING**
> - **WebSearchTool**: $0.025 per call
> - **GPT-5.2 Instant/Thinking**: $1.75/1M input, $14/1M output
> - **GPT-5.2 Pro**: $21/1M input, $168/1M output  
> - **GPT-5.2-Codex**: $2.5/1M input, $20/1M output
> - **GPT-4o (Alternative)**: ~$2.5/1M input, $10/1M output
> - **Estimated total lab cost**: $3-$5

## Prerequisites

Before starting, ensure you have:
- ✅ OpenAI API key with credits
- ✅ Python 3.10+
- ✅ Required packages: `openai>=1.54.0`, `agents`, `python-dotenv`, `pydantic>=2.0`

## What You'll Learn

This comprehensive notebook covers:
1. **Foundation**: GPT-5.2 models, cost tracking, environment setup
2. **Basics**: Simple agents, model comparison, instructions
3. **Tools**: Custom function tools, WebSearchTool, orchestration
4. **Structured Outputs**: Pydantic models, validation, patterns
5. **Multi-Agent Systems**: Handoffs, debates, orchestration
6. **Advanced Features**: Streaming, parallel execution, Pro features
7. **Real-World Use Cases**: RAG, data analysis, content pipelines
8. **Production Patterns**: Error handling, testing, optimization

## Installation

```bash
pip install openai>=1.54.0 agents python-dotenv pydantic
```

## Setup .env File

Create a `.env` file in your project root:
```
OPENAI_API_KEY=your_openai_api_key_here
```

---

**📘 Let's get started!**

In [ ]:
import os
from dotenv import load_dotenv
from agents import Agent, WebSearchTool, FileSearchTool, trace, Runner, function_tool
from agents.model_settings import ModelSettings
from IPython.display import display, Markdown, HTML, JSON
from pydantic import BaseModel, Field
from typing import Optional, List, Dict
import asyncio
import json
from datetime import datetime
import time

# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("❌ OPENAI_API_KEY not found in environment")

print("✅ API key loaded successfully")

# Cost tracking class for GPT-5.2 and GPT-4o
class CostTracker:
    """Track API costs for GPT-5.2 and GPT-4o models"""
    
    PRICING = {
        # GPT-5.2 models
        "gpt-5.2-chat-latest": {"input": 1.75/1_000_000, "output": 14/1_000_000},
        "gpt-5.2": {"input": 1.75/1_000_000, "output": 14/1_000_000},
        "gpt-5.2-pro": {"input": 21/1_000_000, "output": 168/1_000_000},
        "gpt-5.2-codex": {"input": 2.5/1_000_000, "output": 20/1_000_000},
        # GPT-4o as alternative
        "gpt-4o": {"input": 2.5/1_000_000, "output": 10/1_000_000},
        "gpt-4o-mini": {"input": 0.15/1_000_000, "output": 0.6/1_000_000},
    }
    
    def __init__(self):
        self.calls = []
        self.total_cost = 0
        self.web_searches = 0
    
    def add_call(self, model: str, input_tokens: int, output_tokens: int):
        """Track an API call"""
        pricing = self.PRICING.get(model, self.PRICING["gpt-4o"])
        cost = (input_tokens * pricing["input"]) + (output_tokens * pricing["output"])
        
        self.calls.append({
            "model": model,
            "input_tokens": input_tokens,
            "output_tokens": output_tokens,
            "cost": cost,
            "timestamp": datetime.now()
        })
        self.total_cost += cost
    
    def add_web_search(self, count: int = 1):
        """Track web search costs"""
        self.web_searches += count
        self.total_cost += (count * 0.025)
    
    def report(self):
        """Display cost summary"""
        print(f"\n{'='*60}")
        print("💰 COST SUMMARY")
        print(f"{'='*60}")
        print(f"Total API calls: {len(self.calls)}")
        print(f"Web searches: {self.web_searches}")
        print(f"Total cost: ${self.total_cost:.4f}")
        
        if self.calls:
            by_model = {}
            for call in self.calls:
                model = call["model"]
                by_model[model] = by_model.get(model, 0) + call["cost"]
            
            print("\nBy model:")
            for model, cost in by_model.items():
                print(f"  {model}: ${cost:.4f}")
        
        print(f"{'='*60}\n")

tracker = CostTracker()
print("📊 Cost tracker initialized")

## 📍 PHASE 1: Foundation & Setup

## GPT-5.2 Model Family (Released December 2025)

| Model | API ID | Best For | Speed | Input Cost | Output Cost |
|-------|--------|----------|-------|------------|-------------|
| **Instant** | `gpt-5.2-chat-latest` | Fast everyday tasks, simple queries | ⚡⚡⚡ | $1.75/1M | $14/1M |
| **Thinking** | `gpt-5.2` | Complex reasoning, analysis, coding | ⚡⚡ | $1.75/1M | $14/1M |
| **Pro** | `gpt-5.2-pro` | Maximum quality, hardest problems | ⚡ | $21/1M | $168/1M |
| **Codex** | `gpt-5.2-codex` | Agentic coding workflows | ⚡⚡ | $2.5/1M | $20/1M |

### GPT-4o as Alternative

| Model | API ID | Best For | Speed | Input Cost | Output Cost |
|-------|--------|----------|-------|------------|-------------|
| **GPT-4o** | `gpt-4o` | General purpose, multimodal | ⚡⚡ | $2.5/1M | $10/1M |
| **GPT-4o-mini** | `gpt-4o-mini` | Fast, cost-effective | ⚡⚡⚡ | $0.15/1M | $0.6/1M |

---

### When to Use Each Model

**GPT-5.2 Instant** (`gpt-5.2-chat-latest`)
- ✅ Quick Q&A, translations, summaries
- ✅ High-volume simple requests  
- ✅ Fast classification tasks
- 🎯 **Performance**: Fastest response times

**GPT-5.2 Thinking** (`gpt-5.2`)
- ✅ Complex analysis and reasoning
- ✅ Code generation and debugging
- ✅ Research and synthesis
- ✅ Multi-step problem solving
- 🎯 **Performance**: 30% fewer errors vs GPT-5.1

**GPT-5.2 Pro** (`gpt-5.2-pro`)
- ✅ Advanced mathematics
- ✅ Scientific research
- ✅ Critical decision-making
- ✅ Maximum quality requirements
- 🎯 **Performance**: 93.2% GPQA Diamond, 100% AIME 2025

**GPT-5.2-Codex** (`gpt-5.2-codex`)
- ✅ Terminal automation
- ✅ Multi-file code changes
- ✅ Complex refactoring
- 🎯 **Performance**: 56.4% SWE-Bench Pro, 64% Terminal-Bench

**GPT-4o (Alternative)**
- ✅ Use when GPT-5.2 unavailable
- ✅ Similar capabilities to GPT-5.2 Thinking
- ✅ Lower cost than GPT-5.2
- 🎯 **Cost-effective alternative**

In [ ]:
def recommend_model(task_description: str) -> tuple[str, str]:
    """Recommend optimal model for a task (GPT-5.2 or GPT-4o fallback)"""
    
    task_lower = task_description.lower()
    
    # Pro indicators
    if any(kw in task_lower for kw in ["critical", "research", "mathematics", "prove", "scientific"]):
        return "gpt-5.2-pro", "🏆 Maximum quality for critical work (or gpt-4o if unavailable)"
    
    # Codex indicators  
    if any(kw in task_lower for kw in ["code", "programming", "debug", "refactor", "terminal"]):
        return "gpt-5.2-codex", "💻 Optimized for coding (or gpt-4o if unavailable)"
    
    # Thinking indicators
    if any(kw in task_lower for kw in ["analyze", "plan", "reasoning", "complex", "multi-step"]):
        return "gpt-5.2", "🧠 Best for reasoning (or gpt-4o if unavailable)"
    
    # Default to Instant
    return "gpt-5.2-chat-latest", "⚡ Fast for simple tasks (or gpt-4o-mini if unavailable)"

# Test recommendations
print("🎯 Model Recommendation Examples:\n")

tasks = [
    "Translate this text to Spanish",
    "Analyze this 50-page research paper",
    "Solve this advanced calculus problem",
    "Debug my Python code",
    "Quick FAQ answer"
]

for task in tasks:
    model, reason = recommend_model(task)
    print(f"Task: {task}")
    print(f"  → Recommended: {model}")
    print(f"  → {reason}\n")

## What is an AI Agent?

An **Agent** is an autonomous entity that can:
1. 📝 Receive instructions (system prompt)
2. 🛠️ Use tools to gather information
3. 🧠 Reason about tasks
4. ✅ Take actions to complete objectives

### Agent Components

```python
Agent(
    name="AgentName",            # Identifier for tracking
    instructions="What to do",   # System prompt defining behavior
    model="gpt-5.2",            # Which model to use
    tools=[...],                # Available functions (optional)
    output_type=Schema,         # Structured response (optional)
    model_settings=Settings,    # Temperature, etc. (optional)
    handoff_to=[...]           # Other agents (optional)
)
```

### Agent Execution Flow

```
User Input
    ↓
Agent Receives Task
    ↓
Processes with Instructions
    ↓
Calls Tools (if needed)
    ↓
Returns Response
    ↓
Logged to Trace
```

### Key Concepts

- **Tools**: Functions the agent can call (WebSearch, custom functions)
- **Structured Outputs**: Typed responses using Pydantic models
- **Handoffs**: Transferring to other specialized agents
- **Traces**: Execution logs in OpenAI console
- **Model Settings**: Temperature, token limits, tool choice

### Model Selection

- Use `gpt-5.2-chat-latest` (Instant) for simple, fast tasks
- Use `gpt-5.2` (Thinking) for complex reasoning
- Use `gpt-5.2-pro` (Pro) for maximum quality
- Use `gpt-4o` as cost-effective alternative

## Model Performance Benchmarks

### GPT-5.2 Pro Benchmarks
- 📊 **GPQA Diamond**: 93.2%
- 📊 **AIME 2025**: 100%
- 📊 **SWE-Bench Verified**: 80%

### GPT-5.2 Thinking
- 📊 **30% fewer errors** than GPT-5.1 Thinking
- 📊 **70.9% better** than top professionals on GDP

val tasks

### GPT-5.2-Codex
- 📊 **SWE-Bench Pro**: 56.4%
- 📊 **Terminal-Bench 2.0**: 64.0%

### Context & Output
- **Context window**: 400K tokens
- **Max output**: 128K tokens
- **Knowledge cutoff**: August 2025

In [ ]:
# Basic cost estimation examples

def estimate_cost(model: str, input_tokens: int, output_tokens: int) -> float:
    """Estimate cost for a given model and token usage"""
    pricing = CostTracker.PRICING.get(model, CostTracker.PRICING["gpt-4o"])
    return (input_tokens * pricing["input"]) + (output_tokens * pricing["output"])

print("💰 Cost Estimation Examples\n")
print("="*70)

# Example scenarios
scenarios = [
    ("Simple query", "gpt-5.2-chat-latest", 100, 200),
    ("Complex analysis", "gpt-5.2", 500, 1000),
    ("Critical research", "gpt-5.2-pro", 1000, 2000),
    ("Code generation", "gpt-5.2-codex", 800, 1500),
    ("GPT-4o alternative", "gpt-4o", 500, 1000),
]

for desc, model, input_tok, output_tok in scenarios:
    cost = estimate_cost(model, input_tok, output_tok)
    print(f"\n{desc}:")
    print(f"  Model: {model}")
    print(f"  Tokens: {input_tok} in / {output_tok} out")
    print(f"  Cost: ${cost:.4f}")

print("\n" + "="*70)
print(f"\n💡 Tip: Use tracker.add_call() after each agent run to track actual costs")

## Understanding Traces & Debugging

**Traces** are execution logs that help you debug and understand agent behavior.

### What Gets Logged
- 📝 Agent instructions and model used
- 🔄 Complete message history
- 🛠️ Tool calls and responses
- ⏱️ Timing and performance metrics
- 💰 Token usage

### Viewing Traces
All traces are automatically logged to:
**https://platform.openai.com/traces**

### Using Traces

```python
# Create named trace
with trace("My Task Description"):
    result = await Runner.run(agent, "query")

# Generate custom trace ID
trace_id = gen_trace_id()
with trace("Task", trace_id=trace_id):
    result = await Runner.run(agent, "query")
```

### Benefits
- ✅ Debug agent behavior
- ✅ Monitor performance
- ✅ Track costs
- ✅ Optimize prompts
- ✅ Share with team

In [ ]:
# Trace ID generation and organization examples

from agents import gen_trace_id

print("🔍 Trace Organization Examples\n")
print("="*70)

# Generate custom trace IDs
trace_ids = {
    "simple_query": gen_trace_id(),
    "complex_analysis": gen_trace_id(),
    "multi_step_task": gen_trace_id()
}

print("\n📋 Generated Trace IDs:")
for task_name, tid in trace_ids.items():
    print(f"  {task_name}: {tid}")

print("\n💡 Usage:")
print("""
# Organized traces by task type
with trace("Simple Query", trace_id=trace_ids['simple_query']):
    result = await Runner.run(agent, "What is 2+2?")

# All related traces will share same ID prefix
# Easy to find and group in OpenAI console
""")

print("="*70)
print("\n✅ Phase 1 setup complete! Ready for Phase 2.")

## ✅ Phase 1 Complete: Foundation & Setup

### What You Learned
- ✅ GPT-5.2 model family (Instant, Thinking, Pro, Codex)
- ✅ GPT-4o as cost-effective alternative
- ✅ Cost tracking with CostTracker class
- ✅ Model selection for different tasks
- ✅ Agent fundamentals and components
- ✅ Performance benchmarks
- ✅ Trace logging and debugging

### Key Takeaways
1. **Model Selection**: Use Instant for speed, Thinking for reasoning, Pro for quality
2. **Cost Management**: Track costs with `tracker.add_call()`
3. **Debugging**: Use traces to understand agent behavior
4. **Alternatives**: GPT-4o available when GPT-5.2 unavailable

### Next Up: Phase 2 - Basic Agents
In the next phase, you'll learn to:
- Create your first AI agent
- Compare models live
- Engineer effective instructions
- Control temperature and creativity

---

**Ready to build your first agent?** → Continue to Phase 2!

## 📍 PHASE 2: Basic Agents

In this phase, you'll create your first AI agents and learn how to:
- Run agents with different models
- Compare GPT-5.2 vs GPT-4o
- Control creativity with temperature
- Write effective instructions

In [ ]:
# Cell 11: Your First Agent with GPT-5.2 or GPT-4o

# Create a simple agent
basic_agent = Agent(
    name="QuickResponder",
    instructions="Answer questions concisely in Singlish style",
    model="gpt-4o"  # Use gpt-4o (or gpt-5.2-chat-latest if available)
)

# Run the agent
with trace("First Agent Run"):
    result = await Runner.run(basic_agent, "Tell me a joke about AI agents lah")
    print("🤖 Agent Response:")
    print(result.final_output)

# Track cost (estimate)
tracker.add_call("gpt-4o", 50, 150)
print("\n" + "="*60)
print("✅ Your first agent ran successfully!")
print("💡 Check traces at: https://platform.openai.com/traces")
print("="*60)

In [ ]:
# Cell 12: Live Model Comparison (GPT-4o vs GPT-5.2)

print("🔬 Comparing GPT-4o and GPT-5.2 (if available)\n")
print("="*70)

task = "Explain how AI agents work in 2 sentences"

# Test GPT-4o
gpt4o_agent = Agent(
    name="GPT4o-Agent",
    instructions="Explain clearly and concisely",
    model="gpt-4o"
)

print("\n**GPT-4o Response:**")
with trace("GPT-4o Test"):
    result_4o = await Runner.run(gpt4o_agent, task)
    print(result_4o.final_output)
tracker.add_call("gpt-4o", 80, 100)

# Optionally test GPT-5.2 if available
# Uncomment if you have GPT-5.2 access:
# print("\n**GPT-5.2 Response:**")
# gpt52_agent = Agent(
#     name="GPT52-Agent", 
#     instructions="Explain clearly and concisely",
#     model="gpt-5.2-chat-latest"
# )
# with trace("GPT-5.2 Test"):
#     result_52 = await Runner.run(gpt52_agent, task)
#     print(result_52.final_output)
# tracker.add_call("gpt-5.2-chat-latest", 80, 100)

print("\n" + "="*70)
tracker.report()

## Temperature & Creativity Control

**Temperature** controls the randomness/creativity of agent responses:
- **0.0**: Deterministic, consistent, factual
- **0.7**: Balanced (default)
- **1.0+**: More creative, varied, unpredictable

### When to Use Different Temperatures

| Temperature | Best For | Example Use Case |
|-------------|----------|------------------|
| 0.0 - 0.3 | Facts, analysis, code | Math problems, data analysis |
| 0.4 - 0.7 | General purpose | Q&A, instructions |
| 0.8 - 1.2 | Creative content | Stories, marketing copy |
| 1.3 - 2.0 | High creativity | Brainstorming, art |

In [ ]:
# Cell 14: Temperature Comparison Demo

print("🌡️ Temperature Comparison\n")
print("="*70)

task = "Write a one-sentence story opening about robots"

temperatures = [0.0, 0.7, 1.5]

for temp in temperatures:
    print(f"\n**Temperature {temp}:**")
    
    agent = Agent(
        name=f"Agent-temp-{temp}",
        instructions="Write creative story openings",
        model="gpt-4o",
        model_settings=ModelSettings(temperature=temp)
    )
    
    with trace(f"Temp {temp}"):
        result = await Runner.run(agent, task)
        print(result.final_output)
    
    tracker.add_call("gpt-4o", 50, 80)

print("\n" + "="*70)
print("\n💡 Notice: Higher temperature = more creative/varied responses")

## Instruction Engineering Best Practices

Good instructions are the foundation of effective agents. Follow these principles:

### ✅ Good Instructions
- **Specific**: Define exact behavior and output format
- **Clear**: Use simple, unambiguous language
- **Complete**: Include all necessary context
- **Structured**: Use numbered steps or bullet points
- **Examples**: Show desired output format

### ❌ Bad Instructions
- Vague: "Help the user"
- Ambiguous: "Be creative"
- Incomplete: Missing key context
- Unstructured: Wall of text

### Template
```
You are a [ROLE].

Your task:
1. [Step 1]
2. [Step 2]
3. [Step 3]

Output format: [FORMAT]

Example:
[EXAMPLE]
```

In [ ]:
# Cell 16: Good vs Bad Instructions Demo

print("📝 Instruction Quality Comparison\n")
print("="*70)

task = "Summarize this: 'AI agents are autonomous software that can use tools and make decisions.'"

# ❌ Bad: Vague instructions
bad_agent = Agent(
    name="VagueAgent",
    instructions="Help summarize things",
    model="gpt-4o"
)

print("\n❌ **Bad Instructions** ('Help summarize things'):")
result_bad = await Runner.run(bad_agent, task)
print(result_bad.final_output)

# ✅ Good: Specific instructions
good_agent = Agent(
    name="SpecificAgent",
    instructions="""You are a summarization expert.

Task: Summarize text in exactly 1 sentence, max 15 words.
Style: Simple, clear language.
Format: Single sentence, no preamble.""",
    model="gpt-4o"
)

print("\n✅ **Good Instructions** (Specific, structured):")
result_good = await Runner.run(good_agent, task)
print(result_good.final_output)

print("\n" + "="*70)
tracker.add_call("gpt-4o", 100, 100)

## ✅ Phase 2 Complete: Basic Agents

### What You Learned
- ✅ Created first runnable agents with GPT-4o
- ✅ Compared different models
- ✅ Controlled creativity with temperature
- ✅ Engineered effective instructions
- ✅ Good vs bad instruction patterns

### Key Takeaways
1. **Temperature**: 0.0 for facts, 0.7 for balance, 1.5+ for creativity
2. **Instructions**: Specific > Vague, Structured > Unstructured
3. **Model Choice**: GPT-4o for general use, GPT-5.2 when available
4. **Traces**: Always use `with trace()` for debugging

### Next Up: Phase 3 - Custom Tools
Learn to create and use custom function tools to extend agent capabilities!

---

**Ready for Phase 3?** → Continue below!

## 📍 PHASE 3: Custom Tools

Tools extend agent capabilities by giving them access to functions. In this phase, you'll learn:
- Creating custom tools with `@function_tool`
- Using WebSearchTool for web research
- Tool parameter validation
- Multiple tool orchestration

In [ ]:
# Cell 22: Creating Custom Function Tools

@function_tool
def calculate_cost(input_tokens: int, output_tokens: int, model: str = "gpt-4o") -> str:
    """Calculate exact API cost for given model and token usage"""
    pricing = CostTracker.PRICING.get(model, CostTracker.PRICING["gpt-4o"])
    cost = (input_tokens * pricing["input"]) + (output_tokens * pricing["output"])
    return f"💰 Cost for {model}: ${cost:.6f} ({input_tokens} in + {output_tokens} out tokens)"

@function_tool
def get_current_time() -> str:
    """Get current date and time"""
    return f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

@function_tool
def word_count(text: str) -> str:
    """Count words in text"""
    count = len(text.split())
    return f"📊 Word count: {count}"

print("✅ Created 3 custom function tools:")
print("  1. calculate_cost() - Estimates API costs")
print("  2. get_current_time() - Returns current datetime")
print("  3. word_count() - Counts words in text")
print("\n💡 These tools can now be used by agents!")

In [ ]:
# Cell 23: Agent Using Custom Tools

tool_agent = Agent(
    name="ToolUser",
    instructions="You help users by calling the appropriate tools. Always use tools to get accurate information.",
    model="gpt-4o",
    tools=[calculate_cost, get_current_time, word_count],
    model_settings=ModelSettings(tool_choice="required")  # Force tool use
)

print("🤖 Agent with Custom Tools\n")
print("="*70)

queries = [
    "What time is it now?",
    "How many words are in 'AI agents are transforming software development'?",
    "Calculate cost for 1000 input and 500 output tokens using gpt-4o"
]

for query in queries:
    print(f"\n❓ Query: {query}")
    with trace(f"Tool Query"):
        result = await Runner.run(tool_agent, query)
        print(f"💬 Response: {result.final_output}")
    tracker.add_call("gpt-4o", 100, 150)

print("\n" + "="*70)
print("✅ Agent successfully used custom tools!")

You can check what the AI agent in Traces within the Open AI Console: 
https://platform.openai.com/logs?api=traces

## WebSearchTool Deep Dive

**WebSearchTool** is a hosted tool that lets agents search the web for current information.

### OpenAI Hosted Tools
- **WebSearchTool**: Search the web ($0.025 per call)
- **FileSearchTool**: Query Vector Stores
- **ComputerTool**: Automate computer tasks

### WebSearchTool Configuration

```python
WebSearchTool(
    search_context_size="low"    # low, medium, high
)
```

| Context Size | Cost | Use When |
|--------------|------|----------|
| **low** | $ | Simple facts |
| **medium** | $$ | Detailed research |
| **high** | $$$ | Comprehensive analysis |

### Important: Cost Warning
- **$0.025 per search call**
- Can add up quickly ($2-$3 for this lab)
- Use `tracker.add_web_search()` to monitor

### Best Practices
1. Use `tool_choice="required"` to ensure web search
2. Start with "low" context size
3. Track costs with `tracker.add_web_search()`
4. Combine multiple queries when possible

In [ ]:
# Cell 25: Web Search Agent with GPT-4o

SEARCH_INSTRUCTIONS = """You are a research assistant. Search the web and provide:
1. Brief summary (2-3 sentences)
2. Key findings (3-4 bullet points)
3. Source information

Be concise and factual."""

search_agent = Agent(
    name="SearchAgent",
    instructions=SEARCH_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o",  # Or gpt-5.2 if available
    model_settings=ModelSettings(tool_choice="required")
)

# Execute search
print("🔍 Web Search Demo\n")
print("="*70)

query = "Latest AI agent frameworks 2026"
print(f"\nSearching: {query}\n")

with trace("Web Search"):
    result = await Runner.run(search_agent, query)
    display(Markdown(result.final_output))

# Track costs
tracker.add_web_search(1)  # $0.025
tracker.add_call("gpt-4o", 300, 500)

print("\n" + "="*70)
tracker.report()

## ✅ Phase 3 Complete: Custom Tools

### What You Learned
- ✅ Created custom tools with `@function_tool` decorator
- ✅ Used WebSearchTool for web research
- ✅ Forced tool usage with `tool_choice="required"`
- ✅ Tracked web search costs
- ✅ Combined multiple tools in one agent

### Key Takeaways
1. **Custom Tools**: Use `@function_tool` for any Python function
2. **WebSearchTool**: Costs $0.025 per call - track carefully
3. **Tool Choice**: `required` forces tool use, `auto` lets agent decide
4. **Best Practice**: Always track costs with `tracker`

### Tool Types
- **Custom**: Your Python functions
- **Hosted**: WebSearchTool, FileSearchTool, ComputerTool
- **Future**: Build complex tool ecosystems

### Next Up: Phase 4 - Structured Outputs
Learn to get typed, validated responses using Pydantic models!

---

**Ready for Phase 4?** → Continue below!

## 📍 PHASE 4: Structured Outputs

**Structured Outputs** use Pydantic models to get typed, validated responses instead of free-form text.

### Why Structured Outputs?
- ✅ **Type-safe**: Guaranteed data types
- ✅ **Validated**: Automatic validation
- ✅ **Parseable**: Easy to use programmatically
- ✅ **Self-documenting**: Schema describes expected output

### How It Works

```python
# 1. Define Pydantic model
class MyOutput(BaseModel):
    field1: str = Field(description="What this field is")
    field2: int = Field(description="Another field")

# 2. Use as output_type
agent = Agent(
    model="gpt-4o",
    output_type=MyOutput  # Agent must return this structure
)

# 3. Get typed response
result = await Runner.run(agent, "query")
output = result.final_output  # MyOutput instance
```

### Use Cases
- Research reports
- Data extraction
- Classification
- Multi-step plans
- Structured analysis

In [ ]:
# Cell 37: Basic Structured Output Example

# Define output schema
class ResearchPlan(BaseModel):
    topic: str = Field(description="Research topic")
    searches: List[str] = Field(description="3-5 web search queries to perform")
    approach: str = Field(description="Research strategy")
    estimated_time: str = Field(description="Estimated time needed")

# Create agent with structured output
planner_agent = Agent(
    name="ResearchPlanner",
    instructions="Create detailed research plans. Be specific with search queries.",
    model="gpt-4o",
    output_type=ResearchPlan  # Forces this structure
)

# Run agent
print("📋 Structured Output Demo\n")
print("="*70)

topic = "AI Agent security best practices"
print(f"\nPlanning research for: {topic}\n")

with trace("Research Planning"):
    result = await Runner.run(planner_agent, f"Create research plan for: {topic}")
    plan = result.final_output  # ResearchPlan instance

# Access typed fields
print(f"**Topic**: {plan.topic}")
print(f"**Approach**: {plan.approach}")
print(f"**Estimated Time**: {plan.estimated_time}")
print(f"\n**Search Queries**:")
for i, query in enumerate(plan.searches, 1):
    print(f"  {i}. {query}")

print("\n" + "="*70)
print("✅ Got validated, typed output!")

tracker.add_call("gpt-4o", 200, 300)

## ✅ Phase 4 Complete: Structured Outputs

### What You Learned
- ✅ Created Pydantic models for typed outputs
- ✅ Used `output_type` parameter
- ✅ Accessed validated, typed fields
- ✅ Built research planning agent

### Key Takeaways
1. **Pydantic Models**: Define expected structure with `BaseModel`
2. **Field Descriptions**: Help the model understand schema
3. **Type Safety**: Guaranteed data types (str, int, List, etc.)
4. **Validation**: Automatic checking of required fields

### Pattern
```python
class MySchema(BaseModel):
    field: str = Field(description="Clear description")

agent = Agent(output_type=MySchema, ...)
result = await Runner.run(agent, "...")
typed_output = result.final_output  # MySchema instance
```

---

## 🎉 Phases 1-4 Complete!

You've built a strong foundation:
- ✅ Cost tracking and model selection
- ✅ Basic agents with instructions
- ✅ Custom tools and WebSearch
- ✅ Structured, validated outputs

### Next Steps
The remaining phases cover advanced topics:
- **Phase 5**: Multi-agent systems (handoffs, debates)
- **Phase 6**: Advanced features (streaming, parallel execution)
- **Phase 7**: Real-world use cases (RAG, content pipelines)
- **Phases 8-10**: Production patterns, testing, best practices

**📝 Note**: This is a natural checkpoint to save your progress!

---

**Want to continue? Scroll down for advanced phases!**